In [6]:
import requests
import json
import time

# Client credentials
client_id = 'client_56ba263841481d20efa985d60a3b7d58'
client_secret = 'secret_57a6ee7e5a9e1d982431c067585c239d'
scopes = ['api_listings_read']

# Authentication URL
auth_url = 'https://auth.domain.com.au/v1/connect/token'

# List of agency IDs
agency_ids =["35720", "5808", "4740", "20095"]


# Parameters for the GET request
params = {
    'listingStatusFilter': 'liveAndArchived',
    'pageSize': 200  # Fetching 200 listings per page
}

def get_access_token():
    response = requests.post(auth_url, data={
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'client_credentials',
        'scope': ' '.join(scopes),
        'Content-Type': 'application/x-www-form-urlencoded'
    })
    
    json_response = response.json()
    return json_response['access_token']

def fetch_agency_data(agency_id, access_token):
    headers = {"Authorization": f"Bearer {access_token}"}
    all_listings = []
    
    # Iterate through pages 100 to 200
    for page_number in range(1, 300):
        params['pageNumber'] = page_number
        url_endpoint = f'https://api.domain.com.au/sandbox/v1/agencies/{agency_id}/listings'

        response = requests.get(url_endpoint, headers=headers, params=params)
        
        if response.status_code == 200:
            listings = response.json()
            if not listings:  # If no listings, stop fetching further pages
                print(f"No more listings found at page {page_number} for agency {agency_id}.")
                break
            all_listings.extend(listings)
            print(f"Fetched {len(listings)} listings from page {page_number} for agency {agency_id}.")
        else:
            print(f"Failed to retrieve data for agency {agency_id} on page {page_number}: {response.status_code} - {response.reason}")
            break
            # Stop if page doesn't exist or there's another error

        # Optional: Sleep between requests to avoid rate limiting
        time.sleep(1)
    
    return all_listings

def main():
    access_token = get_access_token()
    
    for agency_id in agency_ids:
        print(f"Fetching data for agency {agency_id}...")
        listings = fetch_agency_data(agency_id, access_token)
        
        # Store the data (e.g., save to a JSON file)
        with open(f'../data/landing/listings_agency_{agency_id}.json', 'w') as f:
            json.dump(listings, f, indent=2)
        
        print(f"Data for agency {agency_id} saved.\n")

if __name__ == "__main__":
    main()


Fetching data for agency 35720...
Fetched 200 listings from page 1 for agency 35720.
Fetched 200 listings from page 2 for agency 35720.
Fetched 100 listings from page 3 for agency 35720.
No more listings found at page 4 for agency 35720.
Data for agency 35720 saved.

Fetching data for agency 5808...
Fetched 200 listings from page 1 for agency 5808.
Fetched 200 listings from page 2 for agency 5808.
Fetched 200 listings from page 3 for agency 5808.
Fetched 200 listings from page 4 for agency 5808.
Fetched 200 listings from page 5 for agency 5808.
Fetched 200 listings from page 6 for agency 5808.
Fetched 200 listings from page 7 for agency 5808.
Fetched 200 listings from page 8 for agency 5808.
Fetched 200 listings from page 9 for agency 5808.
Fetched 200 listings from page 10 for agency 5808.
Fetched 200 listings from page 11 for agency 5808.
Fetched 200 listings from page 12 for agency 5808.
Fetched 200 listings from page 13 for agency 5808.
Fetched 200 listings from page 14 for agency 

In [5]:
import os
import requests
import json
import time

# Client credentials
client_id = 'client_56ba263841481d20efa985d60a3b7d58'
client_secret = 'secret_57a6ee7e5a9e1d982431c067585c239d'
scopes = ['api_listings_read']

# Authentication URL
auth_url = 'https://auth.domain.com.au/v1/connect/token'

# Directory where the files are stored
json_dir = '../data/landing/'

# List of agency IDs to scrape (new agencies)
agency_ids = ["3709", "22748", "35557", "32563", "15424", "29733", "22398"]
#[, ]
#[, , , ]

# Parameters for the GET request
params = {
    'listingStatusFilter': 'liveAndArchived',
    'pageSize': 200  # Fetching 200 listings per page
}

def get_access_token():
    response = requests.post(auth_url, data={
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'client_credentials',
        'scope': ' '.join(scopes),
        'Content-Type': 'application/x-www-form-urlencoded'
    })
    
    json_response = response.json()
    return json_response['access_token']

def fetch_agency_data(agency_id, access_token):
    headers = {"Authorization": f"Bearer {access_token}"}
    all_listings = []
    
    # Iterate through pages 1 to 300
    for page_number in range(1, 300):
        params['pageNumber'] = page_number
        url_endpoint = f'https://api.domain.com.au/sandbox/v1/agencies/{agency_id}/listings'

        response = requests.get(url_endpoint, headers=headers, params=params)
        
        if response.status_code == 200:
            listings = response.json()
            if not listings:  # If no listings, stop fetching further pages
                print(f"No more listings found at page {page_number} for agency {agency_id}.")
                break
            all_listings.extend(listings)
            print(f"Fetched {len(listings)} listings from page {page_number} for agency {agency_id}.")
        else:
            print(f"Failed to retrieve data for agency {agency_id} on page {page_number}: {response.status_code} - {response.reason}")
            break  # Stop if there's another error

        # Optional: Sleep between requests to avoid rate limiting
        time.sleep(1)
    
    return all_listings

def get_existing_agency_ids(directory):
    """Get a list of agency IDs from the existing JSON files in the directory."""
    existing_files = [f for f in os.listdir(directory) if f.startswith('listings_agency_') and f.endswith('.json')]
    existing_agency_ids = set()

    for file in existing_files:
        try:
            # Extract agency ID, assumes filenames are like: 'listings_agency_<agency_id>_agency_name.json'
            agency_id = file.split('_')[2]
            existing_agency_ids.add(agency_id)
        except IndexError:
            # Handle unexpected file names
            print(f"Could not extract agency ID from {file}")
    
    return existing_agency_ids

def main():
    access_token = get_access_token()
    
    # Get the set of existing agency IDs
    existing_agency_ids = get_existing_agency_ids(json_dir)

    # Filter out agency IDs that already have a file
    new_agency_ids = [agency_id for agency_id in agency_ids if agency_id not in existing_agency_ids]
    
    if not new_agency_ids:
        print("All agencies have already been scraped.")
        return

    for agency_id in new_agency_ids:
        print(f"Fetching data for agency {agency_id}...")
        listings = fetch_agency_data(agency_id, access_token)
        
        # Store the data (e.g., save to a JSON file)
        with open(f'{json_dir}new/listings_agency_{agency_id}.json', 'w') as f:
            json.dump(listings, f, indent=2)
        
        print(f"Data for agency {agency_id} saved.\n")

if __name__ == "__main__":
    main()


Fetching data for agency 3709...
Failed to retrieve data for agency 3709 on page 1: 207 - Multi-Status
Data for agency 3709 saved.

Fetching data for agency 22748...
Fetched 200 listings from page 1 for agency 22748.
Fetched 200 listings from page 2 for agency 22748.
Fetched 200 listings from page 3 for agency 22748.
Fetched 200 listings from page 4 for agency 22748.
Fetched 200 listings from page 5 for agency 22748.
Fetched 200 listings from page 6 for agency 22748.
Fetched 200 listings from page 7 for agency 22748.
Fetched 200 listings from page 8 for agency 22748.
Fetched 200 listings from page 9 for agency 22748.
Fetched 200 listings from page 10 for agency 22748.
Fetched 192 listings from page 11 for agency 22748.
No more listings found at page 12 for agency 22748.
Data for agency 22748 saved.

Fetching data for agency 35557...
Fetched 200 listings from page 1 for agency 35557.
Fetched 181 listings from page 2 for agency 35557.
No more listings found at page 3 for agency 35557.
Da